In [1]:
import requests
import json
import datetime as dt
import pandas as pd
import sqlite3
from multiprocessing import Pool, cpu_count

In [2]:
# connect to sql database that stores all the values

conn = sqlite3.connect('reddit_data.db')

In [3]:
# function to create a string type output from a list of keywords we want 

def keywords_filter_str(keywords_filter):
    str_kf = ""
    for k in keywords_filter:
        str_kf = str_kf + k + ","
    str_kf = str_kf[:-1]
    return str_kf

In [4]:
# function to query data from pushshift
def extract_data(submissions_endpoint, size, min_comments, subreddits, keywords_filter, start_date, end_date):

    for subreddit in subreddits:
        # loop over all dates in the given range
        curr_date = start_date
        while curr_date < end_date:

            after_dt = int(curr_date.timestamp())
            before_dt = int((curr_date + dt.timedelta(days=1)).timestamp())
            curr_date += dt.timedelta(days= 1)

            # GET
            req = requests.get(f'{submissions_endpoint}?subreddit={subreddit}&fields={keywords_filter_str(keywords_filter)}&after={after_dt}&before={before_dt}&num_comments=>{min_comments}&size={size}')
            
            # validate for any errors
            if req.status_code == 200:
                data = json.loads(req.text)['data']
                data_df = pd.DataFrame(data).set_index('id')
                data_df.to_sql('submissions', conn, if_exists='append') # push the data to sql database
            # log all the errors
            else:
                error_df = pd.DataFrame([{'code': req.status_code, 'date': curr_date, 'subreddit': subreddit}])
                error_df.to_sql('errors', conn, if_exists='append')

In [5]:
# query parameters to work with

submissions_endpoint = 'https://api.pushshift.io/reddit/search/submission/'
size = 500
min_comments = 1

subreddits = ['mensrights', 'theredpill', 'genzedong', 'conservative']
keywords_filter= ['author', 'created_utc', 
                    'full_link', 'id', 'is_self', 
                    'is_video', 'num_comments', 
                    'num_crossposts', 'over_18', 
                    'score', 'selftext', 'subreddit', 
                    'subreddit_id', 'url', 'upvote_ratio', 'title']
start_date = dt.datetime(2018, 5, 1)
end_date = dt.datetime(2022, 11, 1)

In [1]:
extract_data(submissions_endpoint, size, min_comments, subreddits, keywords_filter, start_date, end_date)

NameError: name 'extract_data' is not defined